In [12]:
import pandas as pd
import numpy as np

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Flatten, Dense, Dropout, GRU, Embedding
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import time
import warnings

In [13]:
# Uyarıları kapat
warnings.filterwarnings('ignore')
start = time.time()

In [14]:
# Veri setinin yüklenmesi
training = pd.read_csv('/kaggle/input/hepsiburada-csv/hepsiburada.csv')

Y = training['Rating'].values.tolist()  # 1:pozitif, 0:negatif
X = training['Review'].values.tolist()  # Cümleler

In [15]:
# Veri setini bölmek (%25 test, %75 eğitim için)
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=2)


In [16]:
num_words = 100  # Külliyat içinde en sık geçen kaç kelime için tokenleştirme yapılacak
tokenizer = Tokenizer(num_words=num_words)  # num_words parametresi verilmezse tüm dataset kullanılır
tokenizer.fit_on_texts(X)  # Tokenleştirme yapılır, kelimeler integer'a dönüştürülür, en çok geçen en düşük integer'ı alır
# tokenizer bir sözlüktür, key:kelimeler, value:tokenler

In [17]:
# Eğitim ve test verilerinin tokenleştirilmesi
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [18]:
# Padding işlemi
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)  # ortalama+2*std, 59 çıkacak
max_tokens = int(max_tokens)
np.sum(num_tokens < max_tokens) / len(num_tokens)  # Yorumların %kaçı 59 dan az kelime içeriyor=%95.9
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)  # 59'dan küçükse 0 eklenir, büyükse atılır
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [19]:
# Tokenlerden kelime elde etmek
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [20]:
# Eğitim modeli
model = Sequential()
embedding_size = 50  # Vektör sayısı, burada word2vec yada Glove algoritmaları da kullanılabilir
model.add(Embedding(input_dim=num_words, output_dim=embedding_size, input_length=max_tokens, name='embedding_layer'))

In [21]:
# Cuda destekli GPU var ise GRU yerine CuDNNGRU kullanarak eğitim hızlandırabilir

model.add(GRU(units=32, return_sequences=True, dropout=0.1))
model.add(GRU(units=16, return_sequences=True, dropout=0.1))  # GRU layer 16 output verecek, return_sequence=True ard arda output üretmek için
model.add(GRU(units=8, return_sequences=True, dropout=0.1) ) # GRU layer 8 output verecek
model.add(GRU(units=4, dropout=0.1))  # GRU layer 4 output verecek, return_sequence=False, çünkü bir sonraki layer=output
model.add(Dense(1, activation='sigmoid'))  # Dense layer


In [22]:
# optimizer = adam(lr=1e-3)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])  # İki sınıflı sınıflandırıcı
model.fit(np.asarray(x_train_pad), np.asarray(y_train), epochs=2, batch_size=256)  

sonuclar = model.evaluate(np.asarray(x_test_pad), np.asarray(y_test))  # sonuclar[1] Accuracy

Epoch 1/2
714/714 [==============================] - 70s 80ms/step - loss: 0.2272 - accuracy: 0.9435
Epoch 2/2
1903/1903 [==============================] - 26s 13ms/step - loss: 0.2164 - accuracy: 0.9438


In [23]:
y_pred = model.predict(x=x_test_pad[0:1000])  # ilk 10000 veri için tahmin et
y_pred = y_pred.T[0]  # Sütun veriyi vektöre çevir

cls_pred = np.array([1.0 if p > 0.5 else 0.0 for p in y_pred])  # 0.5 treshold, büyükse 1, küçükse 0
cls_true = np.array(y_test[0:1000])  # Gerçek sınıflar

incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

print(len(incorrect))

60


In [24]:
idx = incorrect[0]  # ilk yanlış tahmin
text = x_test[idx]  # bunun texti
y_pred[idx]  # modelin tahmini
cls_true[idx]  # gerçek sınıf

end = time.time()

print("zaman:", end - start)

zaman: 182.14735674858093


In [25]:
# Modeli metin veri ile denemek
tst = ["Berbat bir hisse hiç bulaşmayın pişman oldum",
       "ben zaten uzun vadeciyim", "bugün borsaya alım gelirmi"]
tokens = tokenizer.texts_to_sequences(tst)
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape
print(model.predict(tokens_pad))

[[0.94690865]
 [0.9469081 ]
 [0.9469091 ]]
